# ADS 509 Sentiment Assignment

This notebook holds the Sentiment Assignment for Module 6 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required.

In a previous assignment you put together Twitter data and lyrics data on two artists. In this assignment we apply sentiment analysis to those data sets. If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Blackboard.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it.

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.*


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from string import punctuation

from nltk.corpus import stopwords

sw = stopwords.words("english")

In [ ]:
# Add any additional import statements you need here
#!pip install emoji
import nltk
nltk.download('stopwords')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 7.0 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
# change `data_location` to the location of the folder on your machine.
data_location = "/content/drive/MyDrive/ADS-509-01/Module6/M1 Assignment Data/M1 Results"

# These subfolders should still work if you correctly stored the
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

# Paths to the files
positive_words_path = '/content/drive/MyDrive/ADS-509-01/Module6/positive-words.txt'
negative_words_path = '/content/drive/MyDrive/ADS-509-01/Module6/negative-words.txt'
tidytext_sentiments_path = '/content/drive/MyDrive/ADS-509-01/Module6/tidytext_sentiments.txt'

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A Pandas data frame would work equally well.

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist.




In [ ]:


# Full paths
lyrics_path = os.path.join(data_location, lyrics_folder)
twitter_path = os.path.join(data_location, twitter_folder)

# Function to read lyrics data into a pandas DataFrame
def read_lyrics_data(lyrics_folder):
    lyrics_data = []
    for artist in os.listdir(lyrics_folder):
        artist_folder = os.path.join(lyrics_folder, artist)
        if os.path.isdir(artist_folder):
            for song_file in os.listdir(artist_folder):
                song_path = os.path.join(artist_folder, song_file)
                with open(song_path, 'r', encoding='utf-8') as file:
                    lyrics = file.read()
                    lyrics_data.append({
                        'artist': artist,
                        'song': song_file,
                        'lyrics': lyrics
                    })
    return pd.DataFrame(lyrics_data)



# Read in the data
lyrics_df = read_lyrics_data(lyrics_path)

# Display the DataFrames
print("Lyrics DataFrame:")
print(lyrics_df)

Lyrics DataFrame:
    artist                                    song  \
0     cher                          cher_stars.txt   
1     cher              cher_comeandstaywithme.txt   
2     cher                         cher_pirate.txt   
3     cher                         cher_mylove.txt   
4     cher                      cher_thesedays.txt   
..     ...                                     ...   
415  robyn              robyn_wedancetothebeat.txt   
416  robyn                robyn_notontheinside.txt   
417  robyn  robyn_doyoureallywantmeshowrespect.txt   
418  robyn              robyn_crashandburngirl.txt   
419  robyn                      robyn_robotboy.txt   

                                                lyrics  
0    "Stars"\n\n\n\nI was never one for saying what...  
1    "Come And Stay With Me"\n\n\n\nI'll send away ...  
2    "Pirate"\n\n\n\nHe'll sail on with the summer ...  
3    "My Love"\n\n\n\nWhen I go away\nI know my hea...  
4    "These Days"\n\n\n\nWell I've been out walk

In [ ]:
# Read in the twitter data
# Full path to twitter folder
twitter_path = os.path.join(data_location, twitter_folder)

# Function to read Twitter data into a pandas DataFrame
def read_twitter_data(twitter_folder):
    twitter_data = []
    for file_name in os.listdir(twitter_folder):
        file_path = os.path.join(twitter_folder, file_name)
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                descriptions = file.read().splitlines()
                twitter_data.extend([{'artist': file_name.split('.')[0], 'description': desc} for desc in descriptions])
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding='latin1') as file:
                descriptions = file.read().splitlines()
                twitter_data.extend([{'artist': file_name.split('.')[0], 'description': desc} for desc in descriptions])
    return pd.DataFrame(twitter_data)

# Read in the data
twitter_df = read_twitter_data(twitter_path)

# Display the DataFrame
print("Twitter DataFrame:")
print(twitter_df)

Twitter DataFrame:
                           artist  \
0                                   
1                                   
2                                   
3             cher_followers_data   
4             cher_followers_data   
...                           ...   
8706873  robynkonichiwa_followers   
8706874  robynkonichiwa_followers   
8706875  robynkonichiwa_followers   
8706876  robynkonichiwa_followers   
8706877  robynkonichiwa_followers   

                                               description  
0           Bud1            %                      ...  
1                                                      ...  
2           E   %                                      ...  
3        screen_name\tname\tid\tlocation\tfollowers_cou...  
4           hsmcnp\tCountry Girl\t35152213\t\t1302\t1014\t  
...                                                    ...  
8706873                                          517669236  
8706874                                         

In [15]:
# Read in the positive and negative words and the
# tidytext sentiment. Store these so that the positive
# words are associated with a score of +1 and negative words
# are associated with a score of -1. You can use a dataframe or a
# dictionary for this.

# Read positive words
with open(positive_words_path, 'r') as pos_file:
    positive_words = pos_file.read().splitlines()

# Read negative words
with open(negative_words_path, 'r') as neg_file:
    negative_words = neg_file.read().splitlines()

# Assign scores to the words
word_scores = {word: 1 for word in positive_words}
word_scores.update({word: -1 for word in negative_words})

# Convert the dictionary to a DataFrame
word_scores_df = pd.DataFrame(list(word_scores.items()), columns=['word', 'score'])

# Display the DataFrame
print(word_scores_df.head())

                                                word  score
0  ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...      1
1                                                 ;      -1
2                        ; Opinion Lexicon: Positive      1
3                                                  ;     -1
4  ; This file contains a list of POSITIVE opinio...      1


## Sentiment Analysis on Songs

In this section, score the sentiment for all the songs for both artists in your data set. Score the sentiment by manually calculating the sentiment using the combined lexicons provided in this repository.

After you have calculated these sentiments, answer the questions at the end of this section.


In [ ]:
# your code here

### Questions

Q: Overall, which artist has the higher average sentiment per song?

A: <!-- Your answer here -->

---

Q: For your first artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score?

A: <!-- Your answer here -->

---

Q: For your second artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score?

A: <!-- Your answer here -->

---

Q: Plot the distributions of the sentiment scores for both artists. You can use `seaborn` to plot densities or plot histograms in matplotlib.




## Sentiment Analysis on Twitter Descriptions

In this section, define two sets of emojis you designate as positive and negative. Make sure to have at least 10 emojis per set. You can learn about the most popular emojis on Twitter at [the emojitracker](https://emojitracker.com/).

Associate your positive emojis with a score of +1, negative with -1. Score the average sentiment of your two artists based on the Twitter descriptions of their followers. The average sentiment can just be the total score divided by number of followers. You do not need to calculate sentiment on non-emoji content for this section.

In [ ]:
# your code here

Q: What is the average sentiment of your two artists?

A: <!-- Your answer here -->

---

Q: Which positive emoji is the most popular for each artist? Which negative emoji?

A: <!-- Your answer here -->

